In [ ]:
from lec_utils import *
import lec21_util as util
np.random.seed(23) # For reproducibility.
def sample_from_pop(n=100):
    x = np.linspace(-2, 3, n)
    y = x ** 3 + (np.random.normal(0, 3, size=n))
    return pd.DataFrame({'x': x, 'y': y})
sample_1 = sample_from_pop()
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder, FunctionTransformer, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
import warnings
warnings.simplefilter('ignore')
from IPython.display import Markdown

<div class="alert alert-info" markdown="1">

#### Lecture 21

# Regularization, Gradient Descent

### EECS 398-003: Practical Data Science, Fall 2024

<small><a style="text-decoration: none" href="https://practicaldsc.org">practicaldsc.org</a> • <a style="text-decoration: none" href="https://github.com/practicaldsc/fa24">github.com/practicaldsc/fa24</a></small>
    
</div>

<script type="text/x-mathjax-config">
  MathJax.Hub.Config({
    TeX: {
      extensions: ["color.js"],
      packages: {"[+]": ["color"]},
    }
  });
  </script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js?config=TeX-AMS_HTML"></script>

### Announcements 📣

- The Portfolio Homework has been released! Read all about it [**here**](https://practicaldsc.org/portfolio). It has two due dates:
    - A checkpoint (worth 15 points / 100) is due on **Monday, November 25th** (no slip days!).
    - The full homework is due on **Saturday, December 7th** (no slip days!).
- Homework 10 will be out later this week.
- The [**Grade Report**](https://www.gradescope.com/courses/823979/assignments/5191081) now includes scores and slip days through Homework 8.
- Please help spread the word about this class by submitting an **anonymous** testimony [**here**](https://docs.google.com/forms/d/e/1FAIpQLSd3YpC1N8T4ocSB0lrTb5-7Gyi2Vl3L-1bSzHag5wKMY_ns9g/viewform) 🙏.<br><small>We'll share some of the responses we get on this form at [practicaldsc.org/next](https://practicaldsc.org/next), and in advertisement emails/posts we share with other students.</small>
- Interested in research opportunities in data science? See [**#302 on Ed**](https://edstem.org/us/courses/61012/discussion/5690003).

### Agenda

- Recap: Ridge Regression.
- LASSO.
- Gradient descent.

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
Remember that you can always ask questions anonymously at the link above!

## Recap: Ridge Regression

---

### Motivation: Polynomial regression

- Last class, we fit a degree 25 polynomial model to Sample 1.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sample_1[['x']], sample_1['y'], random_state=23)
px.scatter(x=X_train['x'], y=y_train, title="Sample 1's Training Data")

In [ ]:
# include_bias=False makes sure that PolynomialFeatures 
# doesn't create a new column of all 1s in the design matrix, since
# LinearRegression already makes one.
model = make_pipeline(PolynomialFeatures(25, include_bias=False), LinearRegression())
model.fit(X_train, y_train)

- This degree 25 polynomial is clearly overfit to the training data.

In [ ]:
fig = px.scatter(x=X_train['x'], y=y_train, title="Sample 1's Training Data")
fig.add_trace(go.Scatter(
    x=X_train['x'].sort_values(),
    y=model.predict(X_train.sort_values('x')),
    mode='lines',
    line=dict(width=4),
    name='Fit Polynomial of Degree 25'
))

### Inspecting the fit degree 25 polynomial

- What are the resulting coefficients of the <b><span style="color:#ff7f0f">fit polynomial</span></b>?

In [ ]:
pd.Series(model.named_steps['linearregression'].coef_, index=range(1, 26))

- What does the resulting polynomial actually look like, as an equation?

In [ ]:
# These coefficients are rounded to two decimal places.
# The coefficient on x^25 is not 0.00, but is something very small.
util.display_features(model.named_steps['linearregression'])

- `sklearn` assigned **really large** values to many features.<br><small>This means that if $x$ changes a little, the output is going to change **a lot**. It seems like some of the terms are trying to "cancel" each other out – some have large negative coefficients, some have large positive coefficients.</small>

- Intuition: In general, **the bigger the optimal parameters $w_0^*, w_1^*, ..., w_d^*$ are, the more _overfit_ the model is to the training data.**

### Ridge regression

- **Idea**: In addition to just minimizing mean squared error, what if we could **also** try and prevent large parameter values?<br><small>Maybe this would lead to less overfitting!</small>

- Minimizing mean squared error, with $L_2$ regularization, is called **ridge regression**. The **objective function** for ridge regression is:

$$R_\text{ridge}(\vec{w}) = \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2 \mathbf{+} \underbrace{\lambda \sum_{j = 1}^d w_j^2}_{\text{penalty!}}$$

- Intuition: Instead of just minimizing mean squared error, we balance minimizing mean squared error and a penalty on the size of the fit coefficients, $w_1^*$, $w_2^*$, ..., $w_d^*$.<br><small>We don't regularize the intercept term!</small>

- $\lambda$ is a **hyperparameter**, which we choose through cross-validation.

- The $\vec{w}_\text{ridge}^*$ that minimizes $R_\text{ridge}(\vec{w})$ is not necessarily the same as $\vec{w}_\text{OLS}^*$, which minimizes $R_\text{sq}(\vec{w})$!

### Another interpretation of ridge regression

- As $\lambda$ increases, the penalty on the size of $\vec{w}_\text{ridge}^*$ increases, meaning that each $w_j^*$ inches closer to 0.

- An equivalent way of formulating the ridge regression objective function,

    $$\text{minimize} \:\:\:\: \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2 + \lambda \sum_{j = 1}^d w_j^2$$

    is as a **constrained** optimization problem:
    
    $$\text{minimize} \:\:\:\:\frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2 \text{  such that   } \sum_{j = 1}^d w_j^2 \leq Q$$

- $Q$ and $\lambda$ are **inversely related**: the larger $Q$ is, the less of a penalty we're putting on size of $\vec{w}_\text{ridge}^*$, so the smaller $\lambda$ is.<br><small>The exact relationship between $Q$ and $\lambda$ is outside of the scope of this course, as is the proof of this fact. Take IOE 310!</small>

### Ridge regression, visualized

$$\text{minimize} \:\:\:\:\frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2 \text{  such that   } \sum_{j = 1}^d w_j^2 \leq Q$$

- Intuitively:

    - The **loss surface** for just the mean squared error component is in <span style="color:blue"><b>blue</b></span>.
    - The constraint, $\sum_{j = 1}^d w_j^2 \leq Q$, is in <span style="color:green"><b>green</b></span>.<br><small>The larger $Q$ is, the larger the radius of the ball is.</small>

<center><img src="imgs/ball.png" width=500>(<a href="https://ds100.org/course-notes-su23/cv_regularization/cv_reg.html">source</a>)</center>

- The bigger $Q$ is – so, the smaller $\lambda$ is – the larger the <span style="color:green"><b>green circle</b></span> is!

- The bigger $Q$ is, the larger the range of possible values for $\vec{w}_\text{ridge}^*$ is, and the closer $\vec{w}_\text{ridge}^*$ gets to $\vec{w}_\text{OLS}^*$.

### Finding $\vec{w}_\text{ridge}^*$

- We know that the $\vec{w}_\text{OLS}^*$ that minimizes mean squared error,
    $$R_\text{sq}(\vec{w}) = \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2$$
  is the one that satisfies the normal equations, $X^TX \vec{w} = X^T \vec{y}$.<br><small>Recall, linear regression that minimizes mean squared error, without any other constraints, is called **ordinary least squares (OLS)**.</small>

- Sometimes, $\vec{w}^*_\text{OLS}$ is unique, and sometimes there are infinitely many possible $\vec{w}^*_\text{OLS}$.<br><small>There are infinitely many possible $\vec{w}^*_\text{OLS}$ when the design matrix, $X$, is not full rank! All of these infinitely many solutions minimize mean squared error.</small>

- Which vector $\vec{w}_\text{ridge}^*$ minimizes the ridge regression objective function?

$$R_\text{ridge}(\vec{w}) = \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2 + \lambda \sum_{j = 1}^d w_j^2$$

- It turns out there is **always** a unique solution for $\vec{w}_\text{ridge}^*$, even if $X$ is not full rank. It is:
    $$\vec{w}_\text{ridge}^* = (X^TX + n \lambda I)^{-1} X^T \vec{y}$$
    <br><small>The proof is outside of the scope of the class, and requires vector calculus.</small>

- Since there is **always** a unique solution, ridge regression is often used in the presence of multicollinearity!

### Taking a step back

- $\vec{w}_\text{ridge}^*$ **doesn't** minimize mean squared error – it minimizes a slightly different objective function.

- So, why would we use ever use ridge regression?

### Ridge regression in `sklearn`

- Fortunately, `sklearn` can perform ridge regression for us.

In [ ]:
from sklearn.linear_model import Ridge

- Just to experiment, let's set $\lambda$ to something extremely large and look at the resulting predictions.

In [ ]:
# The name of the lambda hyperparameter in sklearn is alpha.
model_large_lambda = make_pipeline(PolynomialFeatures(25, include_bias=False), 
                                   Ridge(alpha=1000000000000000000000000000))
model_large_lambda.fit(X_train, y_train)

### Visualizing the extremely regularized model

- What do the <b><span style="color:purple">resulting predictions</span></b> look like?

In [ ]:
fig = px.scatter(x=X_train['x'], y=y_train, title="Sample 1's Training Data")
fig.add_trace(go.Scatter(
    x=X_train['x'].sort_values(),
    y=model_large_lambda.predict(X_train.sort_values('x')),
    mode='lines',
    line=dict(width=4, color='purple'),
    name='Extremely Regularized Polynomial of Degree 25'
))
fig.update_layout(width=1000, height=800)

- What do you notice?

In [ ]:
model_large_lambda.named_steps['ridge'].intercept_

In [ ]:
# All 0!
model_large_lambda.named_steps['ridge'].coef_

In [ ]:
y_train.mean()

### Using `GridSearchCV` to choose $\lambda$

- In general, we won't just arbitrarily choose a value of $\lambda$.

- Instead, we'll perform $k$-fold cross-validation to choose the $\lambda$ that leads to predictions that work best on unseen test data.

In [ ]:
hyperparams = {
    'ridge__alpha': 10.0 ** np.arange(-2, 15) # Try 0.01, 0.1, 1, 10, 100, 1000, ... 
}
model_regularized = GridSearchCV(
    estimator=make_pipeline(PolynomialFeatures(25, include_bias=False), Ridge()),
    param_grid=hyperparams,
    scoring='neg_mean_squared_error'
)
model_regularized.fit(X_train, y_train)

- Let's check the optimal $\lambda$ it found!

In [ ]:
model_regularized.best_params_

### Visualizing the regularized degree 25 model

- What do the <b><span style="color:green">resulting predictions</span></b> look like?

In [ ]:
fig = px.scatter(x=X_train['x'], y=y_train, title="Sample 1's Training Data")
fig.add_trace(go.Scatter(
    x=X_train['x'].sort_values(),
    y=model.predict(X_train.sort_values('x')),
    mode='lines',
    line=dict(width=4),
    name='Unregularized Polynomial of Degree 25'
))
fig.add_trace(go.Scatter(
    x=X_train['x'].sort_values(),
    y=model_regularized.predict(X_train.sort_values('x')),
    mode='lines',
    line=dict(width=4, color='green'),
    name='Regularized Polynomial of Degree 25'
))
fig.update_layout(width=1000, height=800)

- It seems that the <b><span style="color:green">regularized polynomial</span></b> is _less_ overfit to the specific noise in the training data than the <b><span style="color:#ff7f0f">unregularized polynomial</span></b>!

- The largest coefficients are all much smaller now, too.
<br><small>The coefficient on $x^{20}$ is 0.000136.</small>

In [ ]:
util.display_features(model_regularized.best_estimator_.named_steps['ridge'], precision=8)

- Note that none of them are exactly 0, but many of them are close!<br><small>This will be important later.</small>

### Tuning multiple hyperparameters at once

- What if we don't want to fix a polynomial degree in advance, and instead want to choose the degree using cross-validation, while also using ridge regression?

- No problem!<br><small>Note that the next cell takes much longer than the previous call to `fit` took, since it needs to try every combination of $\alpha$ and polynomial degree.</small>

In [ ]:
hyperparams = {
    'ridge__alpha': 10.0 ** np.arange(-2, 15),
    'polynomialfeatures__degree': range(1, 26)
}
model_regularized_degree = GridSearchCV(
    estimator=make_pipeline(PolynomialFeatures(include_bias=False), Ridge()),
    param_grid=hyperparams,
    scoring='neg_mean_squared_error'
)
model_regularized_degree.fit(X_train, y_train)

- Now, let's check the optimal $\lambda$ **and** polynomial degree it found!

In [ ]:
model_regularized_degree.best_params_

### Visualizing the regularized degree 3 model

- What do the <b><span style="color:skyblue">resulting predictions</span></b> look like?

In [ ]:
fig = px.scatter(x=X_train['x'], y=y_train, title="Sample 1's Training Data")
fig.add_trace(go.Scatter(
    x=X_train['x'].sort_values(),
    y=model.predict(X_train.sort_values('x')),
    mode='lines',
    line=dict(width=4),
    name='Unregularized Polynomial of Degree 25'
))
fig.add_trace(go.Scatter(
    x=X_train['x'].sort_values(),
    y=model_regularized.predict(X_train.sort_values('x')),
    mode='lines',
    line=dict(width=4, color='green'),
    name='Regularized Polynomial of Degree 25'
))
fig.add_trace(go.Scatter(
    x=X_train['x'].sort_values(),
    y=model_regularized_degree.predict(X_train.sort_values('x')),
    mode='lines',
    line=dict(width=4, color='skyblue'),
    name='Regularized Polynomial of Degree 3'
))
polyfig = fig.update_layout(width=1000, height=800)
polyfig

In [ ]:
util.display_features(model_regularized_degree.best_estimator_.named_steps['ridge'])

Run the cell below to set up the next slide.

In [ ]:
from sklearn.metrics import mean_squared_error
unregularized_train = mean_squared_error(y_train, model.predict(X_train))
unregularized_test = mean_squared_error(y_test, model.predict(X_test))
regularized_lambda_train = mean_squared_error(y_train, model_regularized.predict(X_train))
regularized_lambda_validation = (-model_regularized.cv_results_['mean_test_score']).min()
regularized_lambda_test = mean_squared_error(y_test, model_regularized.predict(X_test))
regularized_lambda_degree_train = mean_squared_error(y_train, model_regularized_degree.predict(X_train))
regularized_lambda_degree_validation = (-model_regularized_degree.cv_results_['mean_test_score']).min()
regularized_lambda_degree_test = mean_squared_error(y_test, model_regularized_degree.predict(X_test))
results_df = pd.DataFrame(index=['training MSE', 'average validation MSE (across all folds)', 'test MSE']).assign(
    unregularized=[unregularized_train, np.nan, unregularized_test],
    regularized_lambda_only=[regularized_lambda_train, regularized_lambda_validation, regularized_lambda_test],
    regularized_lambda_and_degree=[regularized_lambda_degree_train, regularized_lambda_degree_validation, regularized_lambda_degree_test]
)

In [ ]:
reprs = {'unregularized': '<b><span style="color:#ff7f0f">Unregularized (Degree 25)</span></b>',
         'regularized_lambda_only': '<b><span style="color:green">Regularized (Degree 25)<br><small>Used cross-validation to choose $\lambda$</span></b>',
         'regularized_lambda_and_degree': '<b><span style="color:skyblue">Regularized (Degree 3)<br><small>Used cross-validation to choose $\lambda$ and degree</small></span></b>'}

In [ ]:
results_df_str = results_df.to_html()
for rep in reprs:
    results_df_str = results_df_str.replace(rep, reprs[rep])

### Comparing training, validation, and test errors

- Let's compare the training and testing error of the three polynomials below.

In [ ]:
polyfig

In [ ]:
display(HTML(results_df_str))

- It seems that the regularized polynomial, in which we used cross-validation to choose both the regularization penalty $\lambda$ **and** degree, generalizes best to unseen data!

### What's next?

- Could we have chosen a different method of penalizing each $w_j$ other than $w_j^2$?<br><small>We're about to see another option!</small>

- Ridge regression's objective function happened to have a closed-form solution.<br>What if we want to minimize a function that **can't** be minimized by hand?<br><small>We'll talk about how towards the end of lecture!</small>

- Why is it called ridge regression?<br><small>See Homework 10!</small>

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
What questions do you have about ridge regression?

## LASSO

---

### Penalizing large parameters

- The ridge regression objective function,
    $$R_\text{ridge}(\vec{w}) = \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2 + \lambda \sum_{j = 1}^d w_j^2$$
    minimizes mean squared error, **plus** a **squared** penalty on the size of the fit coefficients, $w_1^*, w_2^*, ..., w_d^*$.

- We called the regularization strategy above "$L_2$ regularization." Could we have regularized another way?

- The **LASSO** objective function uses $L_1$ regularization, which penalizes the **absolute value** of each coefficient:

    $$R_\text{LASSO}(\vec{w}) = \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2 + \lambda \sum_{j = 1}^d |w_j| $$

- LASSO stands for "least absolute shrinkage and selection operator."<br><small>We'll make sense of this name shortly.</small>

### LASSO in `sklearn`

- Unlike with ridge regression or ordinary least squares, there is no general closed-form solution for $\vec{w}_\text{LASSO}^*$.

- But, it can be estimated using numerical methods, which `sklearn` uses under-the-hood. Let's test it out.<br><small>More on numerical methods soon!</small>

In [ ]:
from sklearn.linear_model import Lasso

- Let's use LASSO to fit a degree 25 polynomial to Sample 1.<br><small>Here, we'll **fix** the degree, and cross-validate to find $\lambda$.</small>

In [ ]:
hyperparams = {
    'lasso__alpha': 10.0 ** np.arange(-2, 15)
}
model_regularized_lasso = GridSearchCV(
    estimator=make_pipeline(PolynomialFeatures(25, include_bias=False), Lasso()),
    param_grid=hyperparams,
    scoring='neg_mean_squared_error'
)
model_regularized_lasso.fit(X_train, y_train)

- Our cross-validation routine ends up choosing $\lambda = 0.1$, though on its own, this doesn't really tell us anything.

In [ ]:
model_regularized_lasso.best_params_

### Visualizing the regularized degree 25 model, fit with LASSO

- What do the <b><span style="color:red">resulting predictions</span></b> look like, relative to the fit polynomials from earlier in the lecture?

In [ ]:
polyfig.add_trace(go.Scatter(
    x=X_train['x'].sort_values(),
    y=model_regularized_lasso.predict(X_train.sort_values('x')),
    mode='lines',
    line=dict(width=4, color='red'),
    name='Regularized Polynomial of Degree 25, using LASSO'
))

- What do you notice about the coefficients of the polynomial themselves?

In [ ]:
util.display_features(model_regularized_lasso.best_estimator_.named_steps['lasso'], precision=8)

- **Important**: Note that we fit a degree 25 polynomial, but many of the higher-order terms are missing, since their coefficients ended up being 0!<br><small>There's are no $x^{18}, x^{19}, x^{20}, ..., x^{25}$ terms above, and also no $x$ term.</small>

- The <b><span style="color:red">resulting polynomial</span></b> ends up being of degree 17.

<div class="alert alert-danger">

#### Reference Slide
    
### Comparing training, validation, and test errors, again
    
</div>

- Run the cell below to set up the calculations.

In [ ]:
regularized_lasso_train = mean_squared_error(y_train, model_regularized_lasso.predict(X_train))
regularized_lasso_validation = (-model_regularized_lasso.cv_results_['mean_test_score']).min()
regularized_lasso_test = mean_squared_error(y_test, model_regularized_lasso.predict(X_test))
results_df['regularized_lasso'] = [regularized_lasso_train, regularized_lasso_validation, regularized_lasso_test]
results_df_str = results_df.to_html()
reprs['regularized_lasso'] = '<b><span style="color:red">Regularized using LASSO (Degree 25)<br><small>Used cross-validation to choose $\lambda$</small></span></b>'
for rep in reprs:
    results_df_str = results_df_str.replace(rep, reprs[rep])

- How does our <b><span style="color:red">LASSO-regularized polynomial</span></b> compare, in terms of errors, to our earlier polynomials?

In [ ]:
display(HTML(results_df_str))

### When using LASSO, many coefficients are set to 0!

- When using $L_1$ regularization – that is, when performing LASSO – many of the optimal coefficients $w_1^*, w_2^*, ..., w_d^*$ end up being **exactly 0**.

- This was not the case in ridge regression – there, the optimal coefficients were all very small, but none were exactly 0.

In [ ]:
display(Markdown('#### Fit using Ridge:'))
util.display_features(model_regularized.best_estimator_.named_steps['ridge'], precision=8)

- If a feature has a coefficient of 0, it means it's not being used at all in making predictions.

In [ ]:
display(Markdown('#### Fit using LASSO (notice the larger coefficient on $x^3$):'))
util.display_features(model_regularized_lasso.best_estimator_.named_steps['lasso'], precision=8)

- LASSO implicitly performs **feature selection** for us – it automatically tells us which features we don't need to use.<br><small>Here, it told us "don't use $x$, don't use $x^{18}$, don't use $x^{19}$, ..., don't use $x^{25}$, and instead weigh the $x^2$ and $x^3$ terms more."</small>

- This is where the name "least absolute shrinkage and **selection** operator" comes from.

### Why does LASSO encourage sparsity?

- The fact that many of the optimal coefficients – $w_1^*, w_2^*, ..., w_d^*$ – are 0 when performing LASSO is often stated as:

<center><b>LASSO encourages <i>sparsity</i>.</b></center>

- To make sense of this, let's look at the equivalent formulation of LASSO as a **constrained optimization problem**.

    $$\text{minimize} \:\:\:\: \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2 + \lambda \sum_{j = 1}^d | w_j |$$

    is equivalent to:
    
    $$\text{minimize} \:\:\:\:\frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2 \text{  such that   } \sum_{j = 1}^d | w_j | \leq Q$$

- Again, $Q$ and $\lambda$ are **inversely related**: the larger $Q$ is, the less of a penalty we're putting on size of $\vec{w}_\text{LASSO}^*$, so the smaller $\lambda$ is.

### LASSO, visualized

$$\text{minimize} \:\:\:\:\frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2 \text{  such that   } \sum_{j = 1}^d | w_j | \leq Q$$

- Again:
    - The **loss surface** for just the mean squared error component is in <span style="color:blue"><b>blue</b></span>.
    - The constraint, $\sum_{j = 1}^d |w_j| \leq Q$, is in <span style="color:green"><b>green</b></span>.<br><small>The larger $Q$ is, the larger the side length of the diamond is.</small>

<center><img src="imgs/lasso-diamond.png" width=500>(<a href="https://ds100.org/course-notes-su23/cv_regularization/cv_reg.html">source</a>)</center>

- Notice that the <span style="color:green"><b>constraint set</b></span> has clearly defined "corners," which lie on the axes. The axes are where the parameter values, $w_1$ and $w_2$ here, are 0.

- Due to the shape of the constraint set, it's likely that the minimum value of <b><span style="color:blue">mean squared error</span></b>, among all options in the <b><span style="color:green">green diamond</span></b>, will occur at a corner, where some of the parameter values are 0.

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
What questions do you have about LASSO, or regularization in general?

## Example: Commute times

---

### Another example: Commute times

- Last class, **before we learned about regularization**, we used $k$-fold cross-validation to choose between the following five models that predict commute time in `'minutes'`.

<center><img src="imgs/five-pipelines.png" width=900></center>

- The most complicated model, labeled `departure_hour with poly features + day OHE + month OHE + week`, didn't generalize well to unseen data, relative to more simple models.<br><small>At least, not when we used ordinary least squares to train it.</small>

- Let's use ordinary least squares, ridge regression, **and** LASSO to train commute time models, and compare the results.

In [ ]:
df = pd.read_csv('data/commute-times.csv')
df['day_of_month'] = pd.to_datetime(df['date']).dt.day
df['month'] = pd.to_datetime(df['date']).dt.month_name()
df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('minutes', axis=1), df['minutes'], random_state=23)

### Ordinary least squares for commute times

- Since we'll do the feature transformations repeatedly, we'll save them as a single pipeline.

In [ ]:
week_converter = FunctionTransformer(lambda s: 'Week ' + ((s - 1) // 7 + 1).astype(str),
                                     feature_names_out='one-to-one')
day_of_month_transformer = make_pipeline(week_converter, OneHotEncoder(drop='first'))
# Note the include_bias=False once again!
commute_feature_pipe = make_pipeline(
    make_column_transformer(
        (PolynomialFeatures(3, include_bias=False), ['departure_hour']),
        (OneHotEncoder(drop='first', handle_unknown='ignore'), ['day', 'month']),
        (day_of_month_transformer, ['day_of_month']),
    )
)

- First, we'll fit a "vanilla" linear regression model, i.e. one that just minimizes mean squared error, with no regularization.

In [ ]:
commute_model_ols = make_pipeline(commute_feature_pipe, LinearRegression())
commute_model_ols

- There are no hyperparameters to grid search for here, so we'll just fit the model directly.

In [ ]:
commute_model_ols.fit(X_train, y_train)

- We'll keep `commute_model_ols` aside for now, and compare its performance to the fit regularized models in a few moments.

### Ridge regression for commute times

- Again, let's instantiate a Pipeline for the steps we want to execute.

In [ ]:
commute_pipe_ridge = make_pipeline(commute_feature_pipe, Ridge())
commute_pipe_ridge

- Now, since we need to choose the regularization penalty, $\lambda$, we'll fit a `GridSearchCV` instance with a hyperparameter grid.

In [ ]:
lambdas = 10.0 ** np.arange(-10, 15)
hyperparams = {
    'ridge__alpha': lambdas 
}

In [ ]:
commute_model_ridge = GridSearchCV(
    commute_pipe_ridge,
    param_grid = hyperparams,
    scoring='neg_mean_squared_error',
    cv=10
)
commute_model_ridge.fit(X_train, y_train)

- Which $\lambda$ did it choose?<br><small>On its own, this value of $\lambda$ doesn't really tell us anything.</small>

In [ ]:
commute_model_ridge.best_params_

### Aside: average validation error vs. $\lambda$

- How did the average validation MSE change with $\lambda$?<br><small>Here, large values of $\lambda$ mean **less complex models**, not more complex.</small>

In [ ]:
(
    pd.Series(-commute_model_ridge.cv_results_['mean_test_score'], 
              index=np.log10(lambdas))
    .to_frame()
    .reset_index()
    .plot(kind='line', x='index', y=0)
    .update_layout(xaxis_title='$\log(\lambda)$', yaxis_title='Average Validation MSE')
)

### LASSO for commute times

- Let's instantiate a third Pipeline for the steps we want to execute.

In [ ]:
commute_pipe_lasso = make_pipeline(commute_feature_pipe, Lasso())
commute_pipe_lasso

- Now, since we need to choose the regularization penalty, $\lambda$, we'll fit a `GridSearchCV` instance with a hyperparameter grid.

In [ ]:
lambdas = 10.0 ** np.arange(-10, 15)
hyperparams = {
    'lasso__alpha': lambdas 
}

In [ ]:
commute_model_lasso = GridSearchCV(
    commute_pipe_lasso,
    param_grid = hyperparams,
    scoring='neg_mean_squared_error',
    cv=10
)
commute_model_lasso.fit(X_train, y_train)

- Which $\lambda$ did it choose?<br><small>On its own, this value of $\lambda$ doesn't really tell us anything.</small>

In [ ]:
commute_model_lasso.best_params_

Run the cell below to set up the next slide.

In [ ]:
commute_results = pd.concat([
    util.display_commute_coefs(commute_model_ols),
    util.display_commute_coefs(commute_model_ridge.best_estimator_),
    util.display_commute_coefs(commute_model_lasso.best_estimator_)
], axis=1)
commute_results.columns = ['ols', 'ridge', 'lasso']

### Comparing coefficients across models

- What do the resulting coefficients look like in all three models?

In [ ]:
display_df(commute_results, rows=22)

- The coefficients in the OLS model tend to be the largest in magnitude.

- In the ridge model, the coefficients are all generally small, but none are 0.

- In the LASSO model, many coefficients are 0 exactly.

### Comparing training and test errors across models

In [ ]:
model_dict = {'ols': commute_model_ols, 'ridge': commute_model_ridge, 'lasso': commute_model_lasso}
for model in model_dict:
    display(Markdown(f'#### {model.upper()} model for commute times'))
    display(Markdown(f'Training error: {mean_squared_error(y_train, model_dict[model].predict(X_train))}<br>Test error: {mean_squared_error(y_test, model_dict[model].predict(X_test))}'))

- The best-fitting LASSO model seems to have a lower training and testing MSE than the best-fitting ridge model.

- But, in general, sometimes LASSO performs better on unseen data, and sometimes ridge does. Cross-validate!<br><small>Sometimes, machine learning practitioners say "there's no free lunch" – there's no universal always-best technique to use to make predictions, it always depends on the specific data you have.</small>

### Standardize when regularizing!

- As we discussed a few lectures ago, by **standardizing** our features, we bring them all to the same scale.

- Standardizing features in ordinary least squares doesn't change our model's **performance**; rather, it impacts the interpretability of the coefficients.

- But, when regularizing, we're penalizing the sizes of the coefficients, which can be on wildly different scales if the features are on different scales.

$$R_\text{ridge}(\vec{w}) = \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2 \mathbf{+} \underbrace{\lambda \sum_{j = 1}^d w_j^2}_{\text{penalty!}}$$

- So, **when regularizing a linear model, you should standardize the features first**, so the coefficients for all features are on the same scale, and are penalized equally.

In [ ]:
# In other words, commute_feature_pipe should've been this!
make_pipeline(commute_feature_pipe, StandardScaler())

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
What questions do you have about regularization in general?

## Gradient descent intuition

---

### Minimizing empirical risk

- Repeatedly, we've been tasked with **minimizing** the value of empirical risk functions.<br><small>Why? To help us find the **best** model parameters, $h^*$ or $w^*$, which help us make the **best** predictions!</small>

- We've minimized empirical risk functions in various ways.


$$R_\text{sq}(h) = \frac{1}{n} \sum_{i = 1}^n (y_i - h)^2$$

$$\displaystyle R_\text{abs}(w_0, w_1) = \frac{1}{n} \sum_{i = 1}^n |y_i - (w_0 + w_1 x)|$$

$$\displaystyle R_\text{sq}(\vec{w}) = \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2$$

$$\displaystyle R_\text{ridge}(\vec{w}) = \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2 + \lambda \sum_{j = 1}^d w_j^2$$

### Minimizing arbitrary functions

- Assume $f(w)$ is some **differentiable** function.<br><small>For now, we'll assume $f$ takes in a single number, $w$, as input and returns a single number as its output.</small>

- When tasked with minimizing $f(w)$, our general strategy has been to:<br>
    1. Find $\frac{df}{dw}(w)$, the derivative of $f$.
    2. Find the input $w^*$ such that $\frac{df}{dw}(w^*) = 0$.


- However, there are cases where we can find $\frac{df}{dw}(w)$, but **it is either difficult or impossible to solve $\frac{df}{dw}(w^*) = 0$**.

$$f(w) = 5w^4 - w^3 - 5w^2 + 2w - 9$$

- Then what?

In [ ]:
util.draw_f()

### What does the derivative of a function tell us?

- **Goal**: Given a **differentiable** function $f(w)$, find the input $w^*$ that minimizes $f(w)$.

- What does $\frac{d}{dw} f(w)$ mean?

In [ ]:
from ipywidgets import interact
interact(util.show_tangent, w0=(-1.5, 1.5));

### Let's go hiking!

- Suppose you're at the top of a mountain 🏔️ and need to get **to the bottom**.

- Further, suppose it's really cloudy ☁️, meaning you can only see a few feet around you.

- **How** would you get to the bottom?


<center><img src="imgs/mountain.jpeg" width=500></center>

### Searching for the minimum

- Suppose we're given an initial _guess_ for a value of $w$ that minimizes $f(w)$.

<center>
    
<img src="imgs/positive-slope.png" width=500>
    
</center>

- If the <span style="color:red">**slope of the tangent line at $f(w)$**</span> is **positive** 📈:
    - Increasing $w$ **increases** $f$.
    - This means the minimum must be to the **left** of the point $(w, f(w))$.
    - Solution: **Decrease** $w$ ⬇️.

- The steeper the slope is, the further we must be from the minimum – so, the steeper the slope, the quicker we should decrease $w$!

### Searching for the minimum

- Suppose we're given an initial _guess_ for a value of $w$ that minimizes $f(w)$.

<center>
    
<img src="imgs/negative-slope.png" width=500>
    
</center>

- If the <span style="color:red">**slope of the tangent line at $f(w)$**</span> is **negative** 📉:
    - Increasing $w$ **decreases** $f$.
    - This means the minimum must be to the **right** of the point $(w, f(w))$.
    - Solution: **Increase** $w$ ⬆️.

- The steeper the slope is, the further we must be from the minimum – so, the steeper the slope, the quicker we should increase $w$!

### Intuition

- To minimize $f(w)$, start with an initial guess for the minimizing input, $w^{(0)}$.

- Where do we go next?
    - If $\frac{df}{dw}(w^{(0)}) > 0$, **decrease $w^{(0)}$**.
    - If $\frac{df}{dw}(w^{(0)}) < 0$, **increase $w^{(0)}$**.

- One way to accomplish this:

$$w^{(1)} = w^{(0)} - \frac{df}{dw}(w^{(0)})$$

- A consequence of the above **update rule**: the larger $\frac{df}{dw}$ is, the bigger a step we take!<br><small>This matches our intuition from the previous flew slides – the further we are from the minimum, the bigger of a step we should take!</small>

### Gradient descent

- To minimize a **differentiable** function $f$:
    1. Pick a positive number, $\alpha$. This number is called the **learning rate**, or **step size**.<br><small>Think of $\alpha$ as a hyperparameter of the minimization process.</small>
    2. Pick an **initial guess**, $w^{(0)}$.
    3. Then, repeatedly update your guess using the **update rule**:

$$w^{(t+1)} = w^{(t)} - \alpha \frac{df}{dw}(w^{(i)})$$

<br><br>

- Repeat this process until **convergence** – that is, when $w$ doesn't change much from iteration to iteration.

- This procedure is called **gradient descent**.

### What is gradient descent?

- Gradient descent is a numerical method for finding the input to a function $f$ that minimizes the function.

- It is called **gradient descent** because the gradient is the extension of the derivative to functions of multiple variables.

- A **numerical method** is a technique for approximating the solution to a mathematical problem, often by using the computer.

- Gradient descent is **widely used** in machine learning, to train models from linear regression to neural networks and transformers (includng ChatGPT)!<br><small>In machine learning, we use gradient descent to minimize empirical risk when we can't minimize it by hand, which is true in most, more sophisticated cases.</small>

### Implementing gradient descent

- In practice, we typically don't implement gradient descent ourselves – we rely on existing implementations of it. But, we'll implement it here ourselves to understand what's going on.

- Let's start with an initial guess $w^{(0)} = 0$ and a learning rate $\alpha = 0.01$.

$$w^{(t+1)} = w^{(t)} - \alpha \frac{df}{dw}(w^{(i)})$$

In [ ]:
...

- We see that pretty quickly, $w^{(i)}$ converges to $-0.727$!

### Visualizing $w^{(0)} = 0, \alpha = 0.01$

In [ ]:
util.minimizing_animation(w0=0, alpha=0.01)

### Visualizing $w^{(0)} = 1.1, \alpha = 0.01$

What if we start with a different initial guess?

In [ ]:
util.minimizing_animation(w0=1.1, alpha=0.01)

### Visualizing $w^{(0)} = 0, \alpha = 0.1$

What if we use a different learning rate?

In [ ]:
util.minimizing_animation(w0=0, alpha=0.1)

### Visualizing $w^{(0)} = 0, \alpha = 1$

Some learning rates are so large that the values of $t$ explode towards infinity! Watch what happens when we use a learning rate of 1:

In [ ]:
w = 0
for i in range(50):
    print(round(w, 4), round(util.f(w), 4))
    w = w - 1 * util.df(w)

### Gradient descent and empirical risk minimization

- While gradient descent can minimize other kinds of differentiable functions, its most common use case is in **minimizing empirical risk**.

- For example, consider:
    - The constant model, $H(x) = h$.
    - The dataset $-4, -2, 2, 4$.
    - The initial guess $h_0 = 4$ and the learning rate $\alpha = \frac{1}{4}$.


- **Exercise**: Find $h_1$ and $h_2$.

- See the annotated slides for the solution!

### Lingering questions

- When is gradient descent _guaranteed_ to converge to a global minimum? What kinds of functions work well with gradient descent?

- How do we choose a step size?

- How do we use gradient descent to minimize functions of multiple variables, e.g.:

$$R_\text{ridge}(\vec{w}) = \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2 + \lambda \sum_{j = 1}^d w_j^2$$

- **Question**: Why **can't** we use gradient descent to find $\vec{w}_\text{LASSO}^*$?

$$R_\text{LASSO}(\vec{w}) = \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert^2 + \lambda \sum_{j = 1}^d |w_d|$$